In [1]:
import torch

import gradio as gr
from transformers import pipeline
from transformers.pipelines.audio_utils import ffmpeg_read
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
import uuid
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from InstructorEmbedding import INSTRUCTOR

import tempfile
import os
print(os.environ['PATH'])

MODEL_NAME = "openai/whisper-large-v2"
BATCH_SIZE = 8
FILE_LIMIT_MB = 1000

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)


from pymilvus import connections
from pymilvus import CollectionSchema, FieldSchema, DataType
from pymilvus import utility
from pymilvus import Collection
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Milvus

connections.connect(
  alias="default",
  user='username',
  password='password',
  host='3.14.6.33',
  port='19530'
)

def create_embeddings(input_text):
    model = SentenceTransformer("Muennighoff/SGPT-125M-weightedmean-nli-bitfit")
    embeddings = model.encode(input_text)
    print(embeddings)
    
    return embeddings


def transcribe(inputs, task):
    if inputs is None:
        raise gr.Error("No audio file submitted! Please upload or record an audio file before submitting your request.")
    text = pipe(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=True)["text"]
    print(text)
    return text

input_file = "Samsung Z Fold 5 and Z Flip 5 Impressions They Settled! [TubeRipper.com].m4a"

transcribed_text = transcribe(input_file, "transcribe")
embeddings = create_embeddings(transcribed_text)





/home/cdsw/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/usr/lib/jvm/java-8-openjdk-amd64/bin:/home/cdsw/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/usr/lib/hadoop/bin:/usr/lib/hadoop-hdfs/bin
 So this is the brand new Z Fold 5 and the Z Flip 5. This is what it looks like when Samsung settles. I'll explain. So welcome to your first look at Samsung's newest generation of foldables. They look very similar to last year. Matter of fact, you probably didn't even realize that this is last year's phone. So Samsung makes two foldables right now. There's the Z Flip 5 and the Z Fold 5. Same exact shape and dimensions as last year, same aspect ratios, same cameras, same batteries. It would actually be faster for me to just tell you everything that's different. So that's exactly what I'm gonna do. So I'll start with the Flip 5, the more interesting one. It gets an upgrade to the new Snapdragon 8 Gen 2 chip that's been both powerful and efficient in the S23 series. And it gets a new and improved hinge that's s

In [2]:

access_token = os.environ["HF_TOKEN"]
!huggingface-cli login --token $HF_TOKEN

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

llm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", #meta-llama/Llama-2-13b-chat-hf
                                                     load_in_8bit=True,
                                                     device_map='balanced_low_0',
                                                     torch_dtype=torch.float16,
                                                     low_cpu_mem_usage=True,
                                                     token=access_token
                                                    )
max_len = 4096
llm_task = "text-generation"
#llm_task = "text-generation"
T = 0.1

llm_pipeline = pipeline(
    task=llm_task,
    model=llm_model, 
    tokenizer=tokenizer, 
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.15
)

text_llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Prompt Template for Langchain
template = """You are a helpful AI assistant, please provide a detailed summary for the given context which are meeting notes. Divide your response into a Summary, Key takeaways and Action Items.
Context:{context}
>>Summary<<"""
prompt_template = PromptTemplate(input_variables=["context"], template = template)


text_chain = LLMChain(llm=text_llm, prompt=prompt_template)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/cdsw/.cache/huggingface/token
Login successful


Loading checkpoint shards: 100%|██████████████████| 2/2 [01:03<00:00, 31.81s/it]


In [3]:
sum_text = text_chain(transcribed_text)
print(sum_text["text"])


The summary of the meeting notes is as follows:
* Samsung has launched its latest foldable phones, the Z Flip 5 and the Z Fold 5, with minimal changes compared to last year's models.
* Both phones feature the same aspect ratio, screen resolution, and camera specifications as their predecessors.
* However, the Z Flip 5 receives an upgraded snapdragon chip and a newer hinge design that makes it more durable and easier to use.
* The outside screen of the Z Flip 5 has grown from 1.9 inches to 3.4 inches, providing more readable information and functionality without opening the phone.
* The Z Fold 5 also sees a slight improvement in brightness on its inside display, but otherwise remains unchanged.
* Despite the lack of significant updates, the phones are priced similarly to last year's models, with the Z Flip 5 starting at $1,000 and the Z Fold 5 starting at $1,799.
* The speaker, charging speed, and crease on the screen remain the same as last year's models.
* Samsung has also announced 